# Assignment 2 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [3]:
# Part 1
# Fill in the answer to part 1 below as a decimal number
import numpy as np
P = np.array([[0.3,0.4,0.3],
             [0.2,0.5,0.3],
             [0.4,0.3,0.3]])
# Calculate the two-step transition matrix

two_step_transition = np.linalg.matrix_power(P,2)
# The probability of transitioning from state 2 to state 1 in two steps

probability1 = two_step_transition[1,0]
probability1

0.28

In [4]:
# Part 2
import numpy as np
P = np.array([[0.3,0.4,0.3],
             [0.2,0.5,0.3],
             [0.4,0.3,0.3]])
probability2=probability1-P[1,0]*P[0,0]
probability2


0.22000000000000003

In [ ]:
# Part 3

# Fill in the answer to part 3 below as a boolean
problem1_irreducible = True

In [6]:
# Part 4
import numpy as np

# Define the Transition Matrix for the Markov Chain
P = np.array([[0.3, 0.4, 0.3],
              [0.2, 0.5, 0.3],
              [0.4, 0.3, 0.3]])

# Transpose the Transition Matrix and subtract it from an identity matrix of the same size.
# Transposing is necessary because the standard convention in linear algebra (used by numpy)
# assumes column vectors for state distributions, while Markov chains typically use row vectors.
A = (P.T - np.eye(3))

# Modify the last row of matrix A to be all ones. This step is done to incorporate the constraint
# that the sum of all probabilities in a stationary distribution must equal 1.
A[-1:] = 1

# Define a vector 'b' for the linear equation Ax = b. The last element is set to 1,
# corresponding to the probability constraint, and others are set to 0.
b = np.array([0, 0, 1])

# Solve the linear system Ax = b to find the stationary distribution.
# np.linalg.solve is a function that solves linear systems.
# The result, 'stationary_distribution', is the vector that represents the long-term
# probabilities of being in each state of the Markov Chain.
stationary_distribution = np.linalg.solve(A, b)

# Output the calculated stationary distribution
stationary_distribution


array([0.28888889, 0.41111111, 0.3       ])

In [11]:
# Part 5
# Initialize the expected number of steps from the suburbs and the city to zero.
es = 0
ec = 0

# Set the transition probabilities: 'ess' is the probability of moving from suburbs to suburbs,
# 'esc' is the probability of moving from suburbs to city, and 'ecs' is the probability of moving from city to suburbs.
ess = 0.5
esc = 0.3
ecs = 0.3

# Start a loop to iteratively calculate the expected number of steps. The loop runs 10,000 times to approximate the result.
for _ in range(10000):
    # Calculate the new expected number of steps from the suburbs (es) using the current values of es and ec.
    # This is based on the idea that the expected number of steps from the suburbs is 1 (the current step) plus
    # the weighted average of the expected steps from the next possible states (staying in suburbs or moving to city).
    new_es = 1 + ess * es + esc * ec

    # Similarly, calculate the new expected number of steps from the city (ec) using the current values of es and ec.
    # This accounts for the current step plus the weighted average of the expected steps from the next possible states
    # (moving to suburbs or staying in city).
    new_ec = 1 + esc * es + ecs * ec

    # Update the expected number of steps for both suburbs and city with the newly calculated values.
    es, ec = new_es, new_ec

# After completing the iterations, output the final calculated expected number of steps from the suburbs.
new_es

# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number


3.846153846153845

---
## Assignment 2, PROBLEM 2
Maximum Points = 8


A healthcare organization is interested in understanding the relationship between the number of visits to the doctors office and certain patient characteristics. 
They have collected data on the number of visits (for a year) for a sample of patients and have included the following variables

* ofp : number of physician office visits
* ofnp : number of nonphysician office visits
* opp : number of physician outpatient visits
* opnp : number of nonphysician outpatient visits
* emr : number of emergency room visits
* hosp : number of hospitalizations
* exclhlth : the person is of excellent health (self-perceived)
* poorhealth : the person is of poor health (self-perceived)
* numchron : number of chronic conditions
* adldiff : the person has a condition that limits activities of daily living ?
* noreast : the person is from the north east region
* midwest : the person is from the midwest region
* west : the person is from the west region
* age : age in years (divided by 10)
* male : is the person male ?
* married : is the person married ?
* school : number of years of education
* faminc : family income in 10000$
* employed : is the person employed ?
* privins : is the person covered by private health insurance?
* medicaid : is the person covered by medicaid ?

Decide which patient features are resonable to use to predict the target "number of physician office visits". Hint: should we really use the "ofnp" etc variables?

Since the target variable is counts, a reasonable loss function is to consider the target variable as Poisson distributed where the parameter follows $\lambda = \exp(\alpha \cdot x + \beta)$ where $\alpha$ is a vector (slope) and $\beta$ is a number (intercept). That is, the parameter is the exponential of a linear function. The reason we chose this as our parameter, is that it is always positive which is when the Poisson distribution is defined. To be specific we make the following assumption about our conditional density of $Y \mid X$,
$$
    f_{Y \mid X} (y,x) = \frac{\lambda^{y} e^{-\lambda}}{y !}, \quad \lambda(x) = \exp(\alpha \cdot x + \beta).
$$

Recall from the lecture notes, (4.2) that in this case we should consider the log-loss (entropy) and that according to (4.2.1 Maximum Likelihood and regression) we can consider the conditional log-likelihood. Follow the steps of Example 1 and Example 2 in section (4.2) to derive the loss that needs to be minimized.

Hint: when taking the log of the conditional density you will find that the term that contains the $y!$ does not depend on $\lambda$ and as such does not depend on $\alpha,\beta$, it can thus be discarded. This will be essential due to numerical issues with factorials.

Instructions:

1. Load the file `data/visits_clean.csv`, follow the instructions in the code cell of how this should happen [1.5p]
2. Create the `problem2_X` and the `problem2_y` as numpy arrays with `problem2_X` being the features and `problem2_y` being the target. Do the standard train-test split with 80% training data and 20% testing data. Store these in the variables defined in the cells. [1.5p]
3. Implement $loss$ inside the class `PoissonRegression` by writing down the loss to be minimized, I have provided a formula for the $\lambda$ that you can use. [1.5p]
4. Now use the `PoissonRegression` class to train a Poisson regression model on the training data. [1.5p]
5. Compute the mean absolute error of your prediction on the test set and use Hoeffdings inequality to produce a 95\% confidence interval for the mean absolute error. We can make the assumption that the error is bounded by 70 for simplicity. [2p]


In [80]:
# Part 1

# As in assignment 1 we will load the header into header and data into data
# this time you will have to parse the data such that each data entry is a float
# and that the problem2_data is a numpy array of shape (n_samples,n_columns)
# where n_columns is the number of columns and should have the same length as
# the list of strings header. n_samples is how many rows of data we had.
# If you cannot find the file, check the starting package as it should be updated
# if not, go to the github repo and pull it

# The autograder does not accept pandas as a solution to this problem.
# data/visits_clean.csv
import csv
import numpy as np

# Initialize an empty list to store the data
data = []
header = []
# Open the CSV file and read the data
with open("data/visits_clean.csv", mode = "r") as file:
    # Read the first line for the header
    reader =csv.reader(file, delimiter=' ') 
    header = next(reader)
    for row in reader:
        data.append([float(value) for value in row])
data = np.array(data)


In [81]:
# Assign all columns except the first one as features (X) and the first column as the target (Y) from the 'data' array.
X = data[:, 1:]
Y = data[:, 0]

# Generate an array of indices ranging from 0 to the number of rows in the 'data' array.
indice = np.arange(data.shape[0])

# Shuffle the indices randomly. This randomization is crucial for splitting the data into training and testing sets.
np.random.shuffle(indice)

# Define the ratio of data to be used for training. Here, 80% of the data is set aside for training.
train_ratio = 0.8

# Calculate the number of samples to be included in the training set. The `np.ceil` function ensures that 
# the number is rounded up to the nearest integer.
train_sample = int(np.ceil(data.shape[0] * train_ratio))

# Using the shuffled indices, select the first 80% of the data as the training set for both features (x_train) and target (y_train).
x_train = X[indice[:train_sample]]
y_train = Y[indice[:train_sample]]

# Similarly, select the remaining 20% of the data as the testing set for both features (x_test) and target (y_test).
x_test = X[indice[train_sample:]]
y_test = Y[indice[train_sample:]]





# Split the data into train and randomly using for instance
# np.random.shuffle indices and indexing the first 80% as the train data
# keep the train size as 0.8 rounded up to the nearest integer sample
#problem2_X_train, problem2_X_test, problem2_y_train, problem2_y_test = XXX
x_train

array([[ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 8.,  0.,  0., ...,  0.,  0.,  0.],
       [14.,  0.,  3., ...,  0.,  1.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [82]:
# Part 3

# Fill in the function loss below

class PoissonRegression(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        
        
        # The parameter lambda for the given X and the proposed values 
        # of the coefficients, here coeff[:-1] represent alpha 
        # and coeff[-1] represent beta
        lam = np.exp(np.dot(X,coeffs[:-1])+coeffs[-1])
        neg_loss_function =-np.mean(Y*np.log(lam)-lam)
                # The mean is taken to average the losses over all data points.

        # use the Y variable that is available here to define 
        # the loss function, return the value of the loss for 
        # this Y and for this parameter lam defined above
        return neg_loss_function

    def fit(self,X,Y):
        import numpy as np
        from scipy import optimize

        #Use the loss above together with an optimization method from scipy
        #to find the coefficients of the model
        #this is prepared for you below
        opt_loss = lambda coeffs: self.loss(X,Y,coeffs)
        initial_arguments = np.zeros(shape=X.shape[1]+1) # initial guess as 0
        self.result = optimize.minimize(opt_loss, initial_arguments,method='cg')
        self.coeffs = self.result.x
    
    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            return np.exp(np.dot(X,self.coeffs[:-1])+self.coeffs[-1])

In [83]:
# Part 4

# Initialize your PoissonRegression model
problem2_model = PoissonRegression()

# Fit your initialized model on the training data
problem2_model.fit(x_train,y_train)

# This is to make sure that everything went well, 
# check that success is True
print(problem2_model.result)

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -4.9646743604683605
       x: [ 2.060e-02 -1.239e-03 ...  2.858e-01  1.247e+00]
     nit: 1319
     jac: [-6.616e-06 -9.775e-06 ...  1.788e-07 -7.570e-06]
    nfev: 50072
    njev: 2276


In [98]:
from scipy import optimize
import scipy.special
from sklearn.metrics import mean_absolute_error
import math
prediction = problem2_model.predict(x_test)
actural_value = y_test
# Part 5
MAE= mean_absolute_error(prediction,actural_value)
# Put the computed mean absolute error in the variable below
a= 0
b= 70
alpha =0.05
n= len(y_test)
t = np.sqrt((np.log(2/alpha)*(b-a)**2)/(2*n))

l_edge = max(MAE-t,a)
r_edge = min(MAE+t,b)
problem2_interval=(l_edge, r_edge)
# Put a confidence interval in the variable below by using Hoeffdings inequality using the bounds
# a = 0, b=70 (roughly 5 days between visits as minimum)
# the variable should contain a tuple representing the confidence interval of the form (l_edge,r_edge)
problem2_interval 


(0.75095448862688, 7.15673885768577)

---
## Assignment 2, PROBLEM 3
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [6]:
def problem3_LCG(size=None, seed=0):
    # Initialize an empty list to store the generated pseudo-random numbers.
    out = []

    # Define the modulus M, multiplier a, and increment b as per the Hull-Dobell Theorem.
    M = 2 ** 32
    a = 1664525
    b = 1013904223

    # Initialize the state variable with the seed value.
    state = seed

    # Iterate 'size' times to generate 'size' pseudo-random numbers.
    for _ in range(size):
        # Update the state using the linear congruential generator formula.
        state = (a * seed + b) % M

        # Normalize the state to the range [0, 1) and append to the output list.
        out.append(state / M)

    # Return the list of generated pseudo-random numbers.
    return out


In [7]:

def problem3_uniform(generator=None, period=1, size=None, seed=0):
    # Generate pseudo-random numbers using the provided generator function.
    random_numbers = generator(size=size, seed=seed)

    # Initialize an empty list to store the uniform pseudo-random numbers.
    uniform_random_numbers = []

    # Iterate through the generated pseudo-random numbers.
    for number in random_numbers:
        # Normalize each number to the range [0, 1) based on the generator's period.
        normalized_number = number / period

        # Append the normalized number to the uniform random numbers list.
        uniform_random_numbers.append(normalized_number)

    # Return the list of uniform pseudo-random numbers.
    return uniform_random_numbers


In [8]:
def problem3_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    # Initialize an empty list to store the samples generated from the target distribution.
    samples = []

    # Initialize the current seed value.
    current_seed = seed

    # Define the constant M, the upper bound for the function (pi/2).
    M = np.pi / 2

    # Run the accept-reject algorithm until the desired number of samples is obtained.
    while len(samples) < n_iterations:
        # Generate a pseudo-random number X from the uniform [0,1] distribution.
        X = uniformGenerator(size=1, seed=current_seed)[0]

        # Increment the seed to get a new random number in the next iteration.
        current_seed += 1

        # Compute the function value at X, scaled by pi/2.
        f_x = (np.pi / 2) * abs(np.sin(X * 2 * np.pi))

        # Generate another pseudo-random number u from the uniform [0,1] distribution.
        u = uniformGenerator(size=1, seed=current_seed)[0]

        # Increment the seed again for the next iteration.
        current_seed += 1

        # Check if the generated number u is less than or equal to the scaled function value.
        # If true, accept X as a sample from the target distribution.
        if u <= f_x / M:
            samples.append(X)

    # Return the list of samples generated from the target distribution.
    return samples


---
#### Local Test for Assignment 2, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [9]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem3_LCG(size=10, seed = 1))

period = 2**32

print("Uniform sampler %s" % problem3_uniform(generator=problem3_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem3_uniform(generator=problem3_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem3_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [0.23645552527159452, 0.23645552527159452, 0.23645552527159452, 0.23645552527159452, 0.23645552527159452, 0.23645552527159452, 0.23645552527159452, 0.23645552527159452, 0.23645552527159452, 0.23645552527159452]
Uniform sampler [5.505409214449919e-11, 5.505409214449919e-11, 5.505409214449919e-11, 5.505409214449919e-11, 5.505409214449919e-11, 5.505409214449919e-11, 5.505409214449919e-11, 5.505409214449919e-11, 5.505409214449919e-11, 5.505409214449919e-11]
Accept-Reject sampler [5.505409214449919e-11, 5.523456030661483e-11, 5.5415028468730476e-11, 5.559549663084612e-11, 5.577596479296176e-11, 5.5956432955077406e-11, 5.613690111719305e-11, 5.6317369279308693e-11, 5.6497837441424337e-11, 5.667830560353998e-11, 5.6858773765655624e-11, 5.703924192777127e-11, 5.721971008988691e-11, 5.7400178252002554e-11, 5.75806464141182e-11, 5.776111457623384e-11, 5.7941582738349484e-11, 5.812205090046513e-11, 5.830251906258077e-11, 5.848298722469641e-11]


In [10]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem3_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.23796462709189137, 0.32383276483316237, 0.2590084917154736, 0.6484972199788831, 0.6820045605879779, 0.20985124453651727, 0.38102068999577143, 0.2718754143840908, 0.35184625582788265, 0.243517762709337, 0.22418547484732643, 0.414743999308679, 0.6842409524120733, 0.32375334582076565, 0.27981547357298786, 0.799115369116727, 0.14533616183328069, 0.1995887063541001, 0.14595480067964695, 0.08109066462388448]
